# Bajada de datos de cryptopunks de la DB a csv

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

import requests

import time
import math

import mysql.connector
import random

## Establecer conexión con la BD

In [2]:
# Connect to the new database
mydb = mysql.connector.connect(
  host="localhost",
  user="cecilia",
  password="cecilia",
  database="cryptopunks"
)

mycursor = mydb.cursor()



In [3]:
# Obtain cp and their types
sql ="SELECT `cryptopunks`.`id`, `cryptopunks`.`bids`, `cryptopunks`.`for_sale`, `ctypes`.`name` FROM `cryptopunks`, `ctypes` WHERE `cryptopunks`.`id_type` = `ctypes`.`id`"
mycursor.execute(sql)
por_tipo = mycursor.fetchall()

df_tipo = pd.DataFrame(por_tipo, columns = ['id_cryptopunk', 'bids', 'for_sale', 'type'])
df_tipo = df_tipo.drop_duplicates()

df_tipo.to_csv('cp_y_tipo.csv')
print(df_tipo)

      id_cryptopunk  bids  for_sale   type
0               635     0         1  Alien
1              2890     0         0  Alien
2              3100     0         1  Alien
3              3443     0         0  Alien
4              5822     1         1  Alien
...             ...   ...       ...    ...
7248           7251     0         1   Male
7249           7253     0         0   Male
7250           7254     0         0   Male
7251           7255     0         0   Male
7252           7256     0         0   Male

[7253 rows x 4 columns]


In [4]:
# Obtain all attributes
sql = "SELECT `cryptopunks`.`id`, `cryptopunks`.`bids`, `cryptopunks`.`for_sale`, `attributes`.`name` " \
        "FROM `cryptopunks`, `attributes` , `cryptopunk_has_attributes` " \
        "WHERE `cryptopunks`.`id` = `cryptopunk_has_attributes`.`id_cryptopunk`" \
        "and `cryptopunk_has_attributes`.`id_attribute` = `attributes`.`id`" \
        "ORDER BY `cryptopunks`.`id`"
mycursor.execute(sql)
por_attr = mycursor.fetchall()

df_attr = pd.DataFrame(por_attr, columns = ['id_cryptopunk', 'bids', 'for_sale', 'attribute'])
#df_attr = df_attr.drop_duplicates()
print(df_attr[df_attr.duplicated()])

df_attr.to_csv('cp_y_attr.csv')

print(df_attr) 


Empty DataFrame
Columns: [id_cryptopunk, bids, for_sale, attribute]
Index: []
       id_cryptopunk  bids  for_sale         attribute
0                  0     1         0           Earring
1                  0     1         0        Blonde Bob
2                  0     1         0  Green Eye Shadow
3                  1     0         0             Smile
4                  1     0         0            Mohawk
...              ...   ...       ...               ...
19429           7255     0         0      Nerd Glasses
19430           7255     0         0           Earring
19431           7256     0         0      Nerd Glasses
19432           7256     0         0           Bandana
19433           7256     0         0         Chinstrap

[19434 rows x 4 columns]


In [5]:
# Obtener las transacciones posibles
sql = "SELECT DISTINCT tr_type FROM transactions"

mycursor.execute(sql)
tipo_transac = mycursor.fetchall()

for tr in tipo_transac:
    print(tr) 

('Bid Withdrawn',)
('Bid',)
('Transfer',)
('Sold',)
('Offered',)
('Offer Withdrawn',)
('Claimed',)
('(Unwrap)',)
('(Wrap)',)
('Bid *',)


In [6]:
# Obtener las transacciones posibles
sql = "SELECT transactions.id_cryptopunk, cryptopunks.id_type, transactions.tr_type, transactions.amount, transactions.amount_dols, " \
        "transactions.txn FROM transactions,  cryptopunks WHERE " \
        "transactions.id_cryptopunk = cryptopunks.id "

mycursor.execute(sql)
transac = mycursor.fetchall()

df_transac = pd.DataFrame(transac, columns = ['id_cryptopunk', 'tipo', 'tr_type', 'amount', 'amount_dols', 'date' ])
#df_transac = df_transac.drop_duplicates()
print(df_transac[df_transac.duplicated()])

print(df_transac)

        id_cryptopunk  tipo   tr_type  amount  amount_dols        date
76                635     1       Bid    0.45        129.0  2017-06-27
115              2890     1       Bid    1.17        314.0  2017-07-04
117              2890     1       Bid    0.09         26.0  2017-06-27
159              3100     1       Bid    1.17        314.0  2017-07-04
179              3443     1       Bid    0.00          0.0  2021-04-02
...               ...   ...       ...     ...          ...         ...
128986           7243     5       Bid    0.14         34.0  2017-06-26
129084           7251     5   Offered   56.00     223448.0  2021-12-21
129108           7253     5  Transfer    0.00          0.0  2021-02-19
129123           7254     5  Transfer    0.00          0.0  2021-07-01
129140           7254     5       Bid    1.50       3197.0  2021-06-30

[2040 rows x 6 columns]
        id_cryptopunk  tipo        tr_type  amount  amount_dols        date
0                 635     1  Bid Withdrawn    0

In [7]:
sql = "SELECT * FROM ctypes"

mycursor.execute(sql)
tipos = mycursor.fetchall()

tipos = dict(tipos)
tipos

df_transac['tipo'] = df_transac['tipo'].map(tipos, na_action=None)
df_transac

,id_cryptopunk,tipo,tr_type,amount,amount_dols,date
0,635,Alien,Bid Withdrawn,0.00,0.0,2021-12-04
1,635,Alien,Offered,0.00,0.0,2021-11-10
2,635,Alien,Bid,0.00,0.0,2021-11-10
3,635,Alien,Bid Withdrawn,0.09,415.0,2021-11-08
4,635,Alien,Bid,0.09,411.0,2021-11-08
...,...,...,...,...,...,...
129204,7255,Male,Claimed,0.00,0.0,2017-06-23
129205,7256,Male,Transfer,0.00,0.0,2021-09-12
129206,7256,Male,Sold,25.00,46379.0,2021-02-22
129207,7256,Male,Offered,25.00,4933.0,2020-04-28


In [8]:
df_transac.to_csv('transacciones.csv')

In [ ]:
df_mask = df_transac['tr_type']=='Sold'
df_ventas = df_transac[df_mask]


df_ult_ventas = pd.DataFrame()
ultimo_cp = min(df_ventas['id_cryptopunk'])
max_fecha = min(df_ventas['date'])


# ordeno el df de ventas
df_ventas = df_ventas.sort_values(by=['id_cryptopunk', 'date']).copy()
df_ventas = df_ventas.reset_index(drop=True).copy()

linea_anterior = df_ventas.iloc[0]

for i in df_ventas.index:
    lin = df_ventas.iloc[i]
    if lin['id_cryptopunk'] != linea_anterior['id_cryptopunk']:
        df_ult_ventas = df_ult_ventas.append(linea_anterior)
    linea_anterior = lin
    
    
df_ult_ventas['id_cryptopunk'] =df_ult_ventas[['id_cryptopunk']].astype(int)



In [ ]:
df_ult_ventas.to_csv('ultima_venta.csv')


In [ ]:
df_mask = df_transac['tr_type']=='Offered'
df_ofertas = df_transac[df_mask]


df_ult_ofertas = pd.DataFrame()
ultimo_cp = min(df_ofertas['id_cryptopunk'])
max_fecha = min(df_ofertas['date'])


# ordeno el df de ofertas
df_ofertas = df_ofertas.sort_values(by=['id_cryptopunk', 'date']).copy()
df_ofertas = df_ofertas.reset_index(drop=True).copy()

linea_anterior = df_ofertas.iloc[0]

for i in df_ofertas.index:
    lin = df_ofertas.iloc[i]
    if lin['id_cryptopunk'] != linea_anterior['id_cryptopunk']:
        df_ult_ofertas = df_ult_ofertas.append(linea_anterior)
    linea_anterior = lin
    
    
df_ult_ofertas['id_cryptopunk'] =df_ult_ofertas[['id_cryptopunk']].astype(int)

df_ult_ofertas.to_csv('ultima_oferta.csv')


In [ ]:
df_ult_ofertas

In [ ]:
df_mask = df_attr['for_sale']==1
df_en_venta = df_attr[df_mask]

df_en_venta = df_en_venta.reset_index(drop=True).copy()

df_en_venta

In [ ]:
df_attr_precio = pd.merge(df_en_venta, df_ult_ofertas, on="id_cryptopunk")
df_attr_precio.to_csv('atributo_y_precio.csv')
df_attr_precio